<div dir="rtl">
<font face="XB Zar" size=6>
<h1>مقدمه</h1>
</font>
<font face="XB Zar" size=3>
این تمرین به پیش‌پردازش متن، اصلاح پرسمان، ساخت نمایه، بازیابی boolean و فشرده‌سازی نمایه می‌پردازد.
<br>
دیتاستی که در اختبار شما قرار گرفته است شامل چکیده مقالات و id آن‌ها می‌باشد.
</font>
</div>

In [26]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.metrics import edit_distance
import re
import pandas as pd
from bitarray import bitarray
import os

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/mhdi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mhdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<div dir='rtl'>
<h1> آماده‌سازی دیتاست</h1>
<p>
دیتاستی که در اختیار شما قرار گرفته است، دارای سطر‌هایی می‌باشد که دارای مقدار NaN می‌باشد. برای اینکه بتوانید با این دیتاست کار کنید، باید ابتدا این سطر‌ها را حذف کنید.
</p>
</div>

In [27]:
data_path = "./data/data.csv"
df = pd.read_csv(data_path).dropna()

In [28]:
df.tail(20)

,paperId,abstract
6157,6158,The level of abstraction at which application ...
6158,6159,This paper introduces a novel framework for ex...
6159,6160,Detection of control-flow errors at the softwa...
6160,6161,Recently demonstrated side-channel attacks on ...
6161,6162,Modern Graphics Processing Units (GPUs) widely...
6162,6163,The three dimensional discrete cosine transfor...
6163,6164,This book provides a unified treatment of Flip...
6164,6165,"Smaller feature sizes, reduced voltage levels,..."
6165,6166,"In this paper, we propose a standard micromagn..."
6167,6168,"""Emerging safety-critical real-time control sy..."


<div dir='rtl'>
<h1> Preprocessing (پیش پردازش)</h1>
<p>
بسیاری از داده ‌ها دارای مقادیر زیادی اطلاعات اضافه هستند که در پردازش ها به آن نیازی نیست و یا باعث ایجاد خطا میشوند.
دراین بخش داده را از دیتابیس مورد نظر خوانده
  و سپس پیش پردازش های مورد نیاز را اعمال کنید تا متن پیش پردازش شده را تولید کنید.
پس از اتمام پیش پردازش سایر عملیات گفته شده در ادامه را بر
روی متن ایجاد شده انجام میدهیم.


کلاس
"Preprocessor"
عملیات پیش پردازش را انجام میدهد. نام توابع عمل های مورد نظر نوشته شده است و که با توجه به آن باید کد مخصوص هر یک نوشته شود. تابع
"preprocessor"
تابع اصلی این کلاس است که متن بدون پیش پردازش را گرفته و پردازش های مورد نظر را در آن اعمال میکند و متن مورد نظر را ایجاد میکند.

در این بخش میتوانید از کتابخانه های آماده مانند
<a href="https://www.nltk.org/">NLTK</a>
و
<a href="https://spacy.io/">SpaCy</a>
استفاده کنید.
</p>
</div>

In [29]:
class Preprocessor:

    def __init__(self, stopwords):
        self.stopwords = stopwords
        self.ps = PorterStemmer()


    def preprocess(self, text, is_query=False):
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        tokens = self.word_tokenize(text)
        tokens = self.normalize(tokens)
        if not is_query:
            tokens = self.remove_stopwords(tokens)
        return tokens

    def normalize(self, tokens):
        for i, token in enumerate(tokens):
            tokens[i] = self.ps.stem(token)
        return tokens
    
    def remove_links(self, text):
        text = re.sub('http[s]?://\S+', '', text)        
        return text

    def remove_punctuations(self, text):
        regular_expression = r'[^\w\s]'
        return re.sub(regular_expression, ' ', text.lower())

    def word_tokenize(self, text):
        tokenized_text = nltk.tokenize.word_tokenize(text)
        return tokenized_text

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

preprocessor = Preprocessor(stopwords=stopwords.words('english'))

<div dir='rtl'>
<h1>ساخت نمایه</h1>
<p>
شما در حال توسعه یک موتور جستجوی سریع هستید که از نمایه سازی پویا پشتیبانی می کند. موتور جستجو اسناد جدید را در قالب دسته‌هایی کوچک‌تر   
(batch) هندل می‌کند. در پایان هر روز، این دسته‌ها با استفاده از استراتژی ادغام لگاریتمی ادغام می شوند. هدف به حداقل رساندن هزینه ادغام است.  
مراحلی که باید برای حل این مسئله انجام دهید عبارتند از:
<li>توکن‌بندی و نرمال‌سازی متن از اسناد.</li>
    <li>ایجاد یک index مرتب‌ شده برای هر دسته از اسناد.</li>
    <li>ادغام بهینه چند دسته از indexها با استفاده از یک استراتژی ادغام لگاریتمی.</li>
وظیفه شما این است که بخش‌های خالی <strong>(مشخص شده به‌صورت {TODO})</strong> کد را پر کنید تا موتور جستجو عملی شود.
</p>
</div>

<div dir='rtl'>
<h3> دستورات </h3>
<li>متد <code>sort_based_index_construction</code> از <code>DocumentBatch</code>:</li>
<p>هر سند را با استفاده از تابع‌هایی که در قسمت قبل نوشتید، عمل preprocessing را روی آن انجام دهید.</p>
<p>برای هر توکن، شناسه سند را به فهرست معکوس (inverted index) برای آن توکن اضافه کنید.</p>
<li>متد <code>add_batch</code> در <code>FastSearchEngine</code></li>
<p>فهرست معکوس برای دسته (batch) را ایجاد کنید.</p>
<p>این دسته را به فهرست‌های روزانه اضافه کنید.</p>
<li>متد <code>end_of_day_merge</code> از <code>FastSearchEngine:</code></li>
<p>استراتژی ادغام لگاریتمی را پیاده‌سازی کنید تا به صورت بهینه فهرست‌های روزانه را با فهرست اصلی ادغام کنید.</p>

In [30]:
from collections import defaultdict, deque

class DocumentBatch:
    def __init__(self, docs: dict):
        self.documents = docs
        self.index = defaultdict(set)

    def sort_based_index_construction(self):
        for doc_id, doc in self.documents.items():
            tokens = preprocessor.preprocess(doc)
            for token in tokens:
                self.index[token].add(doc_id)
            
class FastSearchEngine:
    def __init__(self):
        self.main_index = defaultdict(set)
        self.daily_indices = deque()

    def add_batch(self, batch: DocumentBatch):
        batch.sort_based_index_construction()
        self.daily_indices.append(batch.index)
    
    def merge(self, first, second):
        merged_index = defaultdict(set)
        for token in first.keys() | second.keys():
            merged_index[token] = first[token] | second[token]
        return merged_index

    def end_of_day_logarithmic_merge(self):
        while len(self.daily_indices) > 1:
            first = self.daily_indices.popleft()
            second = self.daily_indices.popleft()
            self.daily_indices.append(self.merge(first, second))

        index = self.daily_indices.popleft()
        for token in index:
            main_index_set = self.main_index[token]

            union_set = main_index_set.union(index[token])
            self.main_index[token] = union_set       


In [31]:
# Divide the documents into groups to distribute them between servers. For example, let's consider two servers here.
# Divide the documents of each server into batches, for instance, five batches.
# Create an index for each batch and for each server, then merge them into the main index at the end of the day.
# Repeat this process until all documents are processed.

search_engine = FastSearchEngine()
size = 40
for i in range(size // 4):
    server_a_docs = {int(df.iloc[j]['paperId']): df.iloc[j]['abstract'] for j in range(2 * i, 2 * i + 2)}
    server_b_docs = {int(df.iloc[j]['paperId']): df.iloc[j]['abstract'] for j in range(size // 2 + 2 * i, size // 2 + 2 * i + 2)}
    
    server_a_batch = DocumentBatch(server_a_docs)
    server_b_batch = DocumentBatch(server_b_docs)

    search_engine.add_batch(server_a_batch)
    search_engine.add_batch(server_b_batch)

    search_engine.end_of_day_logarithmic_merge()

print(search_engine.main_index)

defaultdict(<class 'set'>, {'demonstr': {3, 6, 9, 11, 13, 19, 21, 24, 25, 31}, 'end': {28, 21, 31}, 'evalu': {1, 18, 36, 40, 26, 12}, 'kept': {1}, 'system': {1, 5, 6, 9, 10, 11, 12, 14, 15, 17, 20, 21, 23, 24, 27, 28, 29, 36, 37}, 'burden': {21}, 'time': {1, 3, 4, 7, 9, 10, 12, 14, 15, 16, 17, 19, 26, 29, 30, 31, 32, 34, 37}, 'degre': {1, 10, 6}, 'multi': {17, 21, 15}, 'emerg': {16, 1, 20, 12, 30}, 'taken': {17, 21}, 'task': {33, 20, 22, 9, 11}, 'deliv': {17, 2, 12}, 'node': {1, 34, 35, 28, 15}, 'sync': {22}, 'recent': {8, 1, 40, 36}, 'multipl': {2, 3, 39, 24, 25, 15}, 'practic': {2, 12}, '224px': {22}, 'combin': {40, 21}, 'real': {22, 38, 11, 29, 15}, '3': {2, 6, 8, 28, 12}, 'size': {1, 35, 24, 10, 15}, 'year': {1, 14, 17}, 'propos': {1, 3, 7, 8, 9, 10, 12, 13, 14, 17, 20, 22, 23, 27, 29, 30, 32, 36, 38, 39}, 'explor': {2, 34, 38, 24, 26}, 'nois': {21, 38, 39}, 'investig': {18, 35, 21}, 'result': {3, 5, 6, 7, 8, 10, 11, 13, 14, 17, 20, 21, 22, 23, 24, 26, 30, 31, 32, 34, 35, 36, 37, 3

<div dir='rtl'>
<h1>Spell Correction (اصلاح پرسمان)</h1>
<p>
در بسیاری از اوقات پرسمان دادە شده توسط کاربر، ممکن است ناقص یا دارای غلط املایی باشد. برای رفع این مشکل در بسیاری از موتورهای جستجو راە حل هایی تدارک دیده شدە است. ابتدا این راە حل ها را شرح دهید و بیان کنید که یک موتور جست و جو بر چه اساسی پرسمان های اصلاح شده را به کاربر نمایش می دهد.</p>
</div>

<div dir='rtl'>
<h2>پاسخ سوال بالا</h2>
دو نوع اصلاح پرسمان وجود دارد. یکی  isolated word و دیگری contest-sensetive.

حال ما در اینجا فقط از isolated word استفاده می‌کنیم. به این صورت که به ازای تک تک کلمات چک می‌کنیم که اگر آن کلمه در index وجود نداشته یا تعداد بسیار کمی وجود داشت با استفاده از روش k-gram لغات نزدیک به آن کلمه که دارای تعدادی بیشتری gram از حدی که مشخص کرده ایم داشت آن کلمات را به عنوان کاندیدا انتخاب می‌کنیم و سپس با روش edit distance آن کلماتی که ED کمتری داشتند را به کاربر معرفی می‌کنیم.

<div dir='rtl'>
<p>
    در این بخش، ابتدا با استفاده از روش bigram لغات نزدیک به لغات اصلی را پیدا کنید و در آخر با معیار minimum edit distance لغتی جایگزین را برای لغت مورد نظر پیدا کنید.  سپس برای هر پرسمان ورودی کاربر، در صورت اشتباه بودن آن، آن را تصحیح کنید. برای stopword‌ها نیز می‌توانید از لیست موجود که در قسمت‌های قبل ساختید استفاده کنید.
</p>
</div>

In [14]:
from typing import List, Dict


def create_bigram_word(word):
    if len(word) == 0:
        return []

    bigrams = ['$' + word[0]]
    for i in range(len(word)-1):
        bigrams.append(word[i:i+2])
    bigrams.append(word[-1] + '$')
    return bigrams

def create_bigram_index(words: List[str]):
    """
    Creates a bigram index for the spell correction

    Returns
    -------
    dict
        A dictionary of bigrams and their occurence
    """
    bigrams = {}
    for word in words:
        word_bigram = create_bigram_word(word)
        for bigram in word_bigram:
            if bigram not in bigrams:
                bigrams[bigram] = []
            bigrams[bigram].append(word)

    return bigrams

bigram_index = create_bigram_index(search_engine.main_index.keys())
print(bigram_index)

{'$d': ['demonstr', 'degre', 'deliv', 'digit', 'decreas', 'direct', 'devic', 'dataset', 'defect', 'dure', 'dissemin', 'demand', 'divid', 'delay', 'dnn', 'design', 'depend', 'data', 'deep', 'domin', 'dataflow', 'dsp48e', 'differ', 'discuss', 'decis', 'dynam', 'describ', 'decad', 'diod', 'detector', 'determinist', 'dimension', 'desir', 'drive', 'db', 'de', 'datacent', 'defin', 'dodecanethiol', 'definit', 'doubt', 'dram', 'deploy', 'die', 'distribut', 'date', 'detect', 'degrad', 'distanc', 'determin', 'detail', 'due', 'develop', 'daemon', 'deal', 'deepcollis', 'dc', 'dispar', 'deviat', 'dramat', 'dollar', 'derandom', 'densiti', 'decod', 'disk', 'databas', 'divis', 'downsampl', 'diffus', 'dissip', 'directli', 'dncnn', 'diminish', 'denois', 'document', 'display', 'domain', 'defeat', 'dadda', 'difficult'], 'de': ['demonstr', 'burden', 'degre', 'deliv', 'node', 'decreas', 'devic', 'defect', 'demand', 'code', 'delay', 'wide', 'ideal', 'modern', 'design', 'depend', 'model', 'understand', 'ident

In [33]:
def spell_correction(query, bigram_index):
  """
    Correct the give query text, if it is misspelled

    Paramters
    ---------
    query: str
        The query text

    Returns
    ---------
    corrected_query: str
        The corrected text
    """
  query = preprocessor.preprocess(query, is_query=True)
  corrected_query = ""
  for word in query:
    if word in search_engine.main_index or word in preprocessor.stopwords:
      corrected_query += word + " "
      continue

    bigram_word = create_bigram_index([word])
    min_diff = 1000
    max_similarity_word = ""
    for bigram in bigram_word:
      if bigram in bigram_index:
        for word in bigram_index[bigram]:
          diff = edit_distance(word, word)
          if diff < min_diff:
            min_diff = diff
            max_similarity_word = word

    corrected_query += max_similarity_word + " "
  

  return corrected_query

# Example usage
user_query = "Wht is the most populr progarmming lanuage?"
spell_correction(user_query, bigram_index)

'wast is the most practic practic less '

<div dir='rtl'>
<h1> Boolean Retrieval </h1>
<p>
 در این قسمت هدف طراحی یک سامانەی بازیابی اطلاعات boolean می‌باشد. 

برای این کار ابتدا پیش پردازش‌های مورد نیاز را مانند بخش قبل بر روی متون انجام دهید و در مرحله بعد ماتریس doc−term را ایجاد کنید. در نهایت کلاس BooleanRetrievalModel را تکمیل کنید که شامل توابع preprocess_query و find_siⅿiⅼar_docs است که توضیحات هرکدام در قسمت کد موجود است. هدف نهایی این است که هرگاه کوئری به تابع find_siⅿiⅼar_docs از کلاس BooleanRetrievalModel داده شود، شناسه k تا از داک‌هایی که شامل کوئری داده شده هستند برگردانده شوند.
</p>
</div>

In [37]:
class Index:
    def __init__(self, docs: pd.DataFrame):
        self.docs = docs
        self.index = self.create_index()
        self.idf = {}

    def create_index(self):
        index = {}
        for id, doc in self.docs.iterrows():
            tokens = preprocessor.preprocess(doc['abstract'])
            for token in tokens:
                if token not in index:
                    index[token] = []
                if doc['paperId'] not in index[token]:
                    index[token].append(doc['paperId'])
        return index

In [38]:
class BooleanRetrievalModel:
    def __init__(self, doc_term_matrix):
        """    
        Set doc_term_matrix and initialize the model.
        """

        self.index = doc_term_matrix
        self.bigram_index = create_bigram_index(self.index)
    
    def preprocess_query(self, query):
        """
        Do necessary preprocessing here before using the query to find k similar docs.
        Use methods from Preprocess section.
        """
        correct_query = spell_correction(query, self.bigram_index)
        query_tokens = preprocessor.word_tokenize(correct_query)
        query_tokens = preprocessor.remove_stopwords(query_tokens)
        return query_tokens
    
    def find_siⅿiⅼar_docs(self, query, k=20):
        processed_query = self.preprocess_query(query)
        similiar_docs = {}

        for term in processed_query:
            if term not in self.index:
                continue
            for doc in self.index[term]:
                if doc not in similiar_docs:
                    similiar_docs[doc] = 0
                else:
                    similiar_docs[doc] += 1
            
        return sorted(similiar_docs, reverse=True) [:20]

<div dir='rtl'>
در این قسمت ۳ کوئری مختلف به دلخواه خود بزنید و لیست داکیومنت‌های مرتبط با آن‌ها را برگردانید. برای کوتاه‌تر شدن لیست جواب در هر کوئری می‌توانید از عملگر‌های منطقی مانند AND استفاده کنید.
</div>

In [41]:
query_1 = "Wht is the most populr progarmming lanuage?"
query_2 = "how meny language dose exist?"
query_3 = "how can i sort dictionary by value in python"

data_size = 300
doc_index = Index(df.iloc[:data_size,:])
model = BooleanRetrievalModel(doc_index.index)
print(model.find_similar_docs(query_1))
print(model.find_similar_docs(query_2))
print(model.find_similar_docs(query_3))

[302, 299, 295, 293, 291, 290, 289, 288, 287, 286, 282, 277, 275, 273, 272, 271, 269, 265, 261, 259]
[301, 300, 299, 297, 296, 291, 290, 289, 288, 287, 284, 283, 277, 272, 269, 266, 261, 260, 259, 257]
[298, 291, 286, 269, 261, 258, 257, 248, 240, 231, 230, 229, 228, 227, 218, 214, 213, 210, 208, 201]


<div dir='rtl'>

# ذخیره و فشرده‌سازی نمایه
در این بخش، در ابتدا دو الگوریتم فشرده‌سازی gamma code و variable byte را پیاده‌سازی کنید.  
سپس نمایه را به سه شکل زیر ذخیره کنید:
- نمایه‌ی اصلی بدون فشرده‌سازی
- نمایه‌ای که با استفاده از gamma code فشرده شده است.
- نمایه‌ای که با استفاده از variable byte فشرده شده است.

در ادامه اندازه‌ی هر کدام از سه فایل بالا را با استفاده از یک تابع به دست آورده و چاپ کنید.  
همچنین باید تابع‌هایی برای decompress کردن نمایه‌های فشرده‌شده پیاده‌سازی کنید.

**نکته‌ی ۱:** تمامی نمایه‌ها را نیز در کوئرا ارسال کنید. اگر حجم‌شان بیش‌تر از محدودیت کوئرا است، آن‌ها را در یک مکان دیگر آپلود کرده و لینک آن را در این فایل قرار دهید.  
**نکته‌ی ۲:** توابع زیر صرفاً پیشنهادی هستند و هر گونه تغییر تا زمانی که کاربردهای مورد نظر پیاده شود، آزاد است.
</div>

In [2]:
def change_base_number(number, base=2):
    if number == 0:
        return [0]
    digits = []
    while number:
        digits.append(int(number % base))
        number //= base
    return digits[::-1]

def byte_to_binary(data):
    return ''.join(format(x, '08b') for x in data)

def convert_gap_to_real_numbers(gaps):
    last = 0
    result = []
    for gap in gaps:
        last += gap
        result.append(last)
    return result


In [52]:
def gamma_encode(number: int):
    binary = change_base_number(number)
    offset = binary[1:]
    unary = len(offset)
    return [1] * unary + [0] + offset

def gamma_compression(docs, path):
    with open(path, 'wb') as f:
        for word, posting in docs.items():
            previous_docId = 0
            sorted_posting = sorted(posting)
            f.write(bytes(word, encoding='UTF-8'))
            f.write(b'\0')
            line = []
            for docId in sorted_posting:
                line += gamma_encode(docId - previous_docId)
                previous_docId = docId
            line += [0 for i in range(len(line) % 8)]
            arr = bitarray(line)
            f.write(arr.tobytes())
            f.write(b'\0')

        f.write(b'\0')


b'\xa0'


In [53]:
def gamma_decoder_number(encoded: list[str]):
    return int('1'+''.join(encoded), 2)

def gamma_decoder(encoded):
    result = []
    start = 0
    i = 0
    while i < len(encoded):
        if encoded[i] == '0':
            result.append(gamma_decoder_number(encoded[i+1:i + (i-start) + 1]))
            print(encoded[i+1:i + (i-start)+1])
            start = i = i + (i-start) + 1
        i += 1
    return result


def gamma_decompression(path):
    with open(path, 'rb') as f:
        data = f.read()
    
    index = {}
    i = 0
    while True:
        word = ''
        while data[i] != 0:
            word += chr(data[i])
            i += 1
        i += 1
        index[word] = []
        start = i
        while True:
            if data[i] == 0:
                positions = gamma_decoder(bytes(data[start:i]))
                index[word] = convert_gap_to_real_numbers(positions)
                break
            i += 1
        i += 1
        if data[i] == 0 or i > len(data):
            break

    return index

gamma_compression({'hello': [1,2,128, 300]}, './test')
print(gamma_decompression('./test'))

[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]
{'hello': []}


In [28]:
def vb_encode_number(num, block_size):
    result = change_base_number(num, 2 ** (block_size - 1))
    result[-1] += 128
    return bytes(result)

def variable_byte_compression(docs, path, block_size=8):
    with open(path, 'wb') as f:
        for word, posting in docs.items():
            previous_docId = 0
            sorted_posting = sorted(posting)
            f.write(bytes(word, encoding='UTF-8'))
            f.write(b'\0')
            for docId in sorted_posting:
                f.write(vb_encode_number(docId - previous_docId, block_size))
                previous_docId = docId
            f.write(b'\0')

        f.write(b'\0')

In [27]:
def variable_byte_decoder_number(encoded, block_size, is_bit=True):
    bits = encoded
    if not is_bit:
        bits = byte_to_binary(encoded)
    result = ''.join([bits[i + 1:i + block_size] for i in range(0, len(bits), block_size)])
    return int(result, 2)

def variable_byte_decoder(encoded, block_size=8):
    result = []
    start = 0
    bits = byte_to_binary(encoded)
    byte_list = list(encoded)
    for i in range(len(byte_list)):
        if byte_list[i] >= 128:
            result.append(variable_byte_decoder_number(bits[start:(i + 1) * block_size], block_size))
            start = (i + 1) * block_size
    return result


def variable_byte_decompression(path, block_size=8):
    with open(path, 'rb') as f:
        data = f.read()
    
    index = {}
    i = 0
    while True:
        word = ''
        while data[i] != 0:
            word += chr(data[i])
            i += 1
        i += 1
        index[word] = []
        start = i
        while True:
            if data[i] == 0:
                positions = variable_byte_decoder(bytes(data[start:i]), block_size)
                index[word] = convert_gap_to_real_numbers(positions)
                break
            i += 1
        i += 1
        if data[i] == 0 or i > len(data):
            break

    return index

variable_byte_compression({'hello': [1,2,128, 300]}, './test')
print(variable_byte_decompression('./test'))

{'hello': [1, 2, 128, 300]}


In [55]:
NO_COMRESS = "no-compression"
GAMMA = "gamma-code"
VB = "varable-byte"

def save_index(index, path, method):
    if method == NO_COMRESS:
        pass
    if method == GAMMA:
        gamma_compression(index, path)
    if method == VB:
        variable_byte_compression(index, path)

def load_index(path, method):
    if method == NO_COMRESS:
        pass
    if method == GAMMA:
        gamma_decompression(path)
    if method == VB:
        variable_byte_decompression(path)

def get_size(file_path):
    file_stats = os.stat(file_path)
    print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')